# Optimization: principles and algorithms

Bierlaire, M. (2015). *Optimization: Principles and Algorithms.* EPFL Press.

# Part IV: Unconstrained optimization

This notebook replicates the examples from the book, using the python package optimization_book. The numbering of the algorithms, tables and page refer to the book.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import optimization_book.unconstrained as eq

ModuleNotFoundError: No module named 'optimization_book.unconstrained'